# Cross-match ICECAT and 10 year dataset 

In this notebook I cross match the ICECAT and the 10 year (2008 - 2018) datasets released by the IceCube collaboration. This is done because the ICECAT doesn't include the muon energy.

In [91]:
import pandas as pd
import glob
import os
from scipy.spatial import KDTree

In [70]:
icecat = pd.read_csv("IceCube_Gold_Bronze_Tracks.csv")
icecat

,NAME,RUNID,EVENTID,START,EVENTMJD,I3TYPE,RA,DEC,RA_ERR_PLUS,RA_ERR_MINUS,...,ENERGY,FAR,SIGNAL,CASCADE_SCR,SKIMMING_SCR,START_SCR,STOP_SCR,THRGOING_SCR,CR_VETO,OTHER_I3TYPES
0,IC110514A,118178,17334444,2011-05-14 01:32:22.654109,55695.064151,gfu-gold,138.47,-1.94,6.68,3.78,...,187.0,1.30,0.508,3.460000e-07,4.480000e-07,8.110000e-02,4.650000e-05,0.919000,False,NaN
1,IC110610A,118309,46569873,2011-06-10 10:13:33.147086,55722.426078,gfu-gold,272.55,35.64,1.67,2.42,...,294.0,0.25,0.750,3.370000e-10,3.990000e-11,1.200000e-06,1.290000e-04,1.000000,False,gfu-bronze
2,IC110616A,118342,24578488,2011-06-16 17:30:53.939961,55728.729791,gfu-bronze,71.15,5.38,1.41,2.07,...,109.0,4.34,0.257,7.820000e-09,1.320000e-07,5.160000e-05,1.830000e-05,1.000000,False,NaN
3,IC110714A,118435,58198553,2011-07-14 02:42:41.855488,55756.112984,hese-gold,68.20,40.67,0.31,1.10,...,72.0,0.11,0.778,4.330000e-05,1.100000e-05,8.760000e-01,1.810000e-05,0.124000,False,hese-bronze
4,IC110726A,118475,52691508,2011-07-26 12:15:33.258167,55768.510802,gfu-bronze,151.08,6.99,1.19,1.71,...,160.0,1.92,0.396,5.000000e-02,1.390000e-06,9.460000e-01,2.020000e-04,0.003820,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,IC230725A,138193,21103478,2023-07-25 21:30:51.063687,60150.896424,gfu-bronze,327.04,12.33,2.20,2.02,...,145.0,1.68,0.394,2.270000e-07,2.180000e-10,1.000000e+00,3.760000e-09,0.000132,False,NaN
344,IC230727A,138198,44334860,2023-07-27 16:05:39.630288,60152.670598,gfu-bronze,33.66,7.63,1.14,0.70,...,113.0,3.38,0.294,4.880000e-06,9.160000e-10,1.000000e+00,1.080000e-07,0.000053,False,NaN
345,IC230914A,138354,45413430,2023-09-14 05:21:03.717216,60201.222960,gfu-bronze,163.83,31.83,2.55,2.02,...,168.0,0.88,0.544,2.760000e-13,8.770000e-08,3.220000e-08,2.960000e-07,1.000000,False,NaN
346,IC231004A,138415,56188508,2023-10-04 14:39:41.180329,60221.610893,gfu-gold,143.79,25.04,1.05,0.97,...,442.0,0.45,0.842,8.750000e-11,1.440000e-08,1.440000e-04,6.870000e-05,1.000000,False,gfu-bronze


In [106]:
# Only events before 2018
icecat_dropped = icecat.drop(icecat.tail(146).index)

# Verify the result
icecat_dropped

,NAME,RUNID,EVENTID,START,EVENTMJD,I3TYPE,RA,DEC,RA_ERR_PLUS,RA_ERR_MINUS,...,ENERGY,FAR,SIGNAL,CASCADE_SCR,SKIMMING_SCR,START_SCR,STOP_SCR,THRGOING_SCR,CR_VETO,OTHER_I3TYPES
0,IC110514A,118178,17334444,2011-05-14 01:32:22.654109,55695.064151,gfu-gold,138.47,-1.94,6.68,3.78,...,187.0,1.30,0.508,3.460000e-07,4.480000e-07,8.110000e-02,0.000046,0.91900,False,NaN
1,IC110610A,118309,46569873,2011-06-10 10:13:33.147086,55722.426078,gfu-gold,272.55,35.64,1.67,2.42,...,294.0,0.25,0.750,3.370000e-10,3.990000e-11,1.200000e-06,0.000129,1.00000,False,gfu-bronze
2,IC110616A,118342,24578488,2011-06-16 17:30:53.939961,55728.729791,gfu-bronze,71.15,5.38,1.41,2.07,...,109.0,4.34,0.257,7.820000e-09,1.320000e-07,5.160000e-05,0.000018,1.00000,False,NaN
3,IC110714A,118435,58198553,2011-07-14 02:42:41.855488,55756.112984,hese-gold,68.20,40.67,0.31,1.10,...,72.0,0.11,0.778,4.330000e-05,1.100000e-05,8.760000e-01,0.000018,0.12400,False,hese-bronze
4,IC110726A,118475,52691508,2011-07-26 12:15:33.258167,55768.510802,gfu-bronze,151.08,6.99,1.19,1.71,...,160.0,1.92,0.396,5.000000e-02,1.390000e-06,9.460000e-01,0.000202,0.00382,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,IC180417A,130932,35022693,2018-04-17 06:41:03.494152,58225.278513,gfu-gold,305.73,-4.41,3.60,1.58,...,202.0,0.85,0.577,1.810000e-05,1.100000e-06,1.500000e-01,0.000107,0.85000,False,gfu-bronze
198,IC180528A,131096,32665194,2018-05-28 12:09:00.393219,58266.506255,gfu-bronze,312.14,0.30,1.41,2.02,...,110.0,4.20,0.278,1.290000e-06,2.710000e-06,2.700000e-02,0.000016,0.97300,False,NaN
199,IC180608A,131134,60192271,2018-06-08 14:19:31.856250,58277.596896,gfu-bronze,69.08,-1.08,1.63,1.41,...,158.0,2.07,0.396,1.580000e-05,8.900000e-07,1.250000e-03,0.001010,0.99800,False,NaN
200,IC180612A,131145,43542963,2018-06-12 04:34:10.301490,58281.190397,gfu-bronze,338.69,3.73,5.10,5.71,...,107.0,4.66,0.250,2.910000e-07,2.040000e-06,3.930000e-03,0.000044,0.99600,False,NaN


In [93]:
# Define the path to your CSV files
csv_files_path = "IC86_*.csv"  # Update this path to your actual files

# Get a list of all CSV files
csv_files = glob.glob(csv_files_path)

# List to hold individual DataFrames
dataframes = []

# Loop over the list of CSV files and read each one into a DataFrame
for csv_file in csv_files:
    try:
        df = pd.read_csv(csv_file, sep="\s+")
        dataframes.append(df)
    except Exception as e:
        print(f"Error reading {csv_file}: {e}")

# Check if all DataFrames have the same columns
columns = dataframes[0].columns
for df in dataframes:
    if not df.columns.equals(columns):
        print(f"Column mismatch found in {csv_file}. Expected columns: {columns}, but found: {df.columns}")

# Concatenate all DataFrames into a single DataFrame
ic86 = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
ic86

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,56400.507095,4.92,0.20,314.246,-26.519,164.885,63.530
1,56400.529671,4.51,0.43,66.127,-16.144,61.200,73.882
2,56400.529858,4.96,0.79,145.736,-20.735,341.649,69.204
3,56400.540443,5.11,0.20,333.504,-28.945,157.675,61.118
4,56400.548719,5.04,0.27,312.225,-33.462,181.923,56.587
...,...,...,...,...,...,...,...
897401,58307.961110,5.14,0.20,221.246,-59.959,141.239,29.956
897402,58307.962666,2.94,1.23,191.547,4.415,171.627,94.312
897403,58307.963556,2.88,0.30,308.127,42.543,55.437,132.606
897404,58307.963972,2.98,0.21,332.199,44.720,31.489,134.811


In [107]:
# Extract the MJD columns
mjd1 = icecat_dropped['EVENTMJD']
mjd2 = ic86['MJD[days]']

# Find the intersection of the MJD values from both datasets
common_mjd = pd.Series(list(set(mjd1) & set(mjd2)))

# Count the number of matching MJD values
num_matching_mjd = len(common_mjd)
print(f"Number of matching MJD values: {num_matching_mjd}")

Number of matching MJD values: 0


In [108]:
# Extract the Ra and Dec columns
ra_dec1 = icecat_dropped[['RA', 'DEC']]
ra_dec2 = ic86[['RA[deg]', 'Dec[deg]']]

# Create pairs of Ra and Dec
ra_dec1_pairs = list(ra_dec1.itertuples(index=False, name=None))
ra_dec2_pairs = list(ra_dec2.itertuples(index=False, name=None))

# Find the intersection of the Ra and Dec pairs from both datasets
common_ra_dec_pairs = set(ra_dec1_pairs) & set(ra_dec2_pairs)

# Convert the common pairs to a DataFrame
common_ra_dec_df = pd.DataFrame(common_ra_dec_pairs, columns=['Ra', 'Dec'])
common_ra_dec_df

,Ra,Dec


In [111]:
# Extract columns and identifiers for dataset1
ra_dec_mjd1 = icecat_dropped[['NAME', 'RA', 'DEC', 'EVENTMJD', 'RA_ERR_MINUS', 'RA_ERR_PLUS', 'DEC_ERR_MINUS', 'DEC_ERR_PLUS']].to_numpy()

# Extract columns for dataset2
ra_dec_mjd2 = ic86[['RA[deg]', 'Dec[deg]', 'MJD[days]', ]].to_numpy()

# Create KDTree for dataset2 using Ra and Dec
tree = KDTree(ra_dec_mjd2[:, :2])  # Use only Ra and Dec for KDTree

# List to store the results
matches = []

# Iterate over each event in dataset1
for event in ra_dec_mjd1:
    event_id1, ra1, dec1, mjd1, ra_err_minus, ra_err_plus, dec_err_minus, dec_err_plus = event
    tolerance_ra = max(ra_err_minus, ra_err_plus)
    tolerance_dec = max(dec_err_minus, dec_err_plus)
    tolerance = max(tolerance_ra, tolerance_dec)
    
    # Query the KDTree for points within the tolerance
    idx = tree.query_ball_point([ra1, dec1], tolerance)
    
    # If there are matches, further filter by MJD
    for i in idx:
        ra2, dec2, mjd2 = ra_dec_mjd2[i]
        mjd_tolerance = 1  # Set your MJD tolerance; adjust as needed
        if abs(mjd1 - mjd2) <= mjd_tolerance:
            matches.append((event_id1, ra1, dec1, mjd1, ra2, dec2, mjd2))

# Convert matches to a DataFrame for easy viewing
matches_df = pd.DataFrame(matches, columns=['EventID1', 'Ra1', 'Dec1', 'MJD1', 'Ra2', 'Dec2', 'MJD2'])

# Display the matches
matches_df

,EventID1,Ra1,Dec1,MJD1,Ra2,Dec2,MJD2
0,IC110514A,138.47,-1.94,55695.064151,136.502,-2.056,55695.776587
1,IC110514A,138.47,-1.94,55695.064151,137.799,-2.048,55695.064144
2,IC110514A,138.47,-1.94,55695.064151,141.108,-0.213,55695.508692
3,IC110514A,138.47,-1.94,55695.064151,137.467,0.258,55695.278542
4,IC110610A,272.55,35.64,55722.426078,272.503,35.699,55722.426077
...,...,...,...,...,...,...,...
588,IC180613A,38.06,11.53,58282.982360,37.166,11.374,58282.982360
589,IC180613A,38.06,11.53,58282.982360,36.434,13.946,58283.090002
590,IC180613A,38.06,11.53,58282.982360,34.257,15.405,58283.272185
591,IC180613A,38.06,11.53,58282.982360,39.920,8.603,58282.483454


In [113]:
# Define a function to print matches for a specific EventID1
def print_matches_for_event(event_id1):
    filtered_matches = matches_df[matches_df['EventID1'] == event_id1]
    print(filtered_matches)

# Example: Print matches for a specific EventID1
specific_event_id1 = 'IC110514A'  # Replace with the specific EventID1 you want to check
print_matches_for_event(specific_event_id1)

    EventID1     Ra1  Dec1          MJD1      Ra2   Dec2          MJD2
0  IC110514A  138.47 -1.94  55695.064151  136.502 -2.056  55695.776587
1  IC110514A  138.47 -1.94  55695.064151  137.799 -2.048  55695.064144
2  IC110514A  138.47 -1.94  55695.064151  141.108 -0.213  55695.508692
3  IC110514A  138.47 -1.94  55695.064151  137.467  0.258  55695.278542


In [115]:
# Group by EventID1 and count the matches
match_counts = matches_df.groupby('EventID1').size().reset_index(name='count')

# Filter for events with only one match
single_matches = match_counts[match_counts['count'] == 1]['EventID1']

# Display the events from dataset1 that have only one match in dataset2
single_match_events = matches_df[matches_df['EventID1'].isin(single_matches)]
single_match_events

,EventID1,Ra1,Dec1,MJD1,Ra2,Dec2,MJD2
4,IC110610A,272.55,35.64,55722.426078,272.503,35.699,55722.426077
7,IC110714A,68.20,40.67,55756.112984,68.354,40.753,55756.112976
8,IC110726A,151.08,6.99,55768.510802,152.440,6.570,55768.510799
11,IC110818A,332.45,-2.09,55791.688643,332.773,-2.668,55791.688635
15,IC110907A,196.08,9.40,55811.794616,196.574,9.592,55811.794607
...,...,...,...,...,...,...,...
571,IC180228A,294.79,26.40,58177.572404,295.342,26.528,58177.572404
572,IC180313A,287.18,5.53,58190.678580,287.006,5.828,58190.678580
577,IC180410A,218.50,0.56,58218.776795,218.445,0.403,58218.776795
581,IC180528A,312.14,0.30,58266.506255,311.928,0.222,58266.506255
